Exercise 2.1
Gapcost funtion: Definition 8.1.25 in [Ohlebusch] says that a gap penalty function (or what we also refer to as a gapcost function)  must be subadditive, i.e. g(k+l) ≤ g(k) + g(l). What do you think that the reason for this requirement is?

ANSWER: 
Subadditivity (subadditivity is a property of a function that states, roughly, that evaluating the function for the sum of two elements of the domain always returns something less than or equal to the sum of the function's values at each element.)

Biological observations:
gaps are useually longer than one characther but long gaps are less frequent than small gaps
Therefore:
gaps should be considered as single units. Gap costs should depend on the length of the gap, they should be monotonously growing, but not as fast as the legth itself.

if they are not subadditive a gap is cheaper if it is considered as two succesive gaps instead of one combined gap.



STORM ANSWER:

If the gapcost function is subadditive then it is always better (or equally good) to explain a gap of length k as a single event rather than some combination of smaller gaps. This is a desired property when the alignment is interpreted as e.g. reflecting evolution. 

Also if we have a subadditive gapcost function g, then we can immediately compute the cost of this gap/alignment 

agtcgt
------

as g(6), where as, if the gapcost function is not subadditive, then we would need some further information/computation about how to optimally compute the cost as g(k) + g(l), where k+l=6.

Exercise 2.2
Details about the explained algorithm for global pairwise alignment with affine gapcost: Explain why D(i,j)=max{M(i-1,j)-(a+b), I(i-1,j)-(a+b), D(i-1,j)-a} = max{S(i-1,j)-(a+b), D(i-1,j)-a} as stated on slide 9 in the slides about affine gapcost.

ANSWER: 
In the slides about affine gapcost it says:

(*) D(i,j)=max{M(i-1,j)-(a+b), I(i-1,j)-(a+b), D(i-1,j)-a} = max{S(i-1,j)-(a+b), D(i-1,j)-a}.

By definition 

S(i-1,j) = max{M(i-1,j), I(i-1,j), D(i-1,j)},

which means that we can rewrite max{S(i-1,j)-(a+b), D(i-1,j)-a} as

max{M(i-1,j)-(a+b), I(i-1,j)-(a+b), D(i-1,j)-(a+b), D(i-1,j)-a}.

Since b>=0, then D(i-j)-(a+b) <= D(i,j)-a, so the above can be simplified to

max{M(i-1,j)-(a+b), I(i-1,j)-(a+b), D(i-1,j)-a},

which explains the equalities in (*).

Exercise 2.3
An iterative algorithm for computing the optimal cost of a global pairwise alignment with affine gapcost: The slides from the lecture shows a set of recursions, S, D, I, that can be used to compute the optimal cost of a global pairwise alignment with affine gapcost in time O(n*m) when implemented using memorization (dynamic programming). Write pseudocode that compute the three tables, and the cost of a global pairwise alignment with affine gapcost in time O(n*m), in a non-recursive manner.

In [ ]:
import numpy as np
#recursive
def global_alignment_affine_gap_cost(x, y, match_score=1, mismatch_score=-1, gap_open=-1, gap_extend=-1):
    """Returns the global alignment score of x and y using affine gap cost.
    
    Args:
        x: string, the first sequence.
        y: string, the second sequence.
        match_score: int, the score for a match.
        mismatch_score: int, the score for a mismatch.
        gap_open: int, the score for opening a gap.
        gap_extend: int, the score for extending a gap.
        
    Returns:
        int, the global alignment score of x and y.
    """
    # Initialize the matrix.
    n = len(x)
    m = len(y)
    F = np.zeros((n + 1, m + 1))
    E = np.zeros((n + 1, m + 1))
    G = np.zeros((n + 1, m + 1))
    F[0, :] = np.arange(m + 1) * gap_open
    F[:, 0] = np.arange(n + 1) * gap_open
    E[0, :] = np.arange(m + 1) * gap_open
    G[:, 0] = np.arange(n + 1) * gap_open
    # Fill in the matrix.
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            F[i, j] = max(F[i - 1, j - 1] + (match_score if x[i - 1] == y[j - 1] else mismatch_score),
                E[i - 1, j - 1] + (match_score if x[i - 1] == y[j - 1] else mismatch_score),
                          G[i - 1, j - 1] + (match_score if x[i - 1] == y[j - 1] else mismatch_score))
            E[i, j] = max(F[i - 1, j] + gap_open,
                          E[i - 1, j] + gap_extend)
            G[i, j] = max(F[i, j - 1] + gap_open,
                          G[i, j - 1] + gap_extend)
    return F

In [3]:
import numpy as np

#dynamic 
def affine_gap_cost_matrix(seq1, seq2, match_cost, mismatch_cost, start_penaltey, extention_penalty):
    """Return the cost matrix for affine gap alignment.
    
    The cost matrix is a 3D array, where the first two dimensions are
    the same as the cost matrix for global alignment, and the third
    dimension is the gap cost.
    """
    #start the match matrix
    M = np.zeros((len(seq1)+1,len(seq2)+1))  
    for i in range(len(seq1)+1):
        for j in range(len(seq2)+1):
            if i==0 and j==0:
                    M[i,j] = 0
            elif i==0:
                if j == 1: 
                    M[i][j] = start_penaltey
                else: 
                    M[i][j] = M[i][j-1]+extention_penalty
            elif j==0:
                if i==1: 
                    M[i][j] = start_penaltey
                else: 
                    M[i][j] = M[i-1][j]+extention_penalty
            else:
                if seq1[i-1]==seq2[j-1]:
                    M[i,j]=M[i-1,j-1]+match_cost
                else: 
                    M[i,j]=M[i-1,j-1]+mismatch_cost
    # a previous gap in seq1 matrix
    D = np.zeros((len(seq1)+1,len(seq2)+1))
    for i in range(len(seq1)+1):
        for j in range(len(seq2)+1):
            if i==0 and j==0:
                    D[i,j] = 0
            elif i==0:
                if j == 1: 
                    D[i][j] = start_penaltey
                else: 
                    D[i][j] = M[i][j-1]+extention_penalty
            elif j==0:
                if i==1: 
                    D[i][j] = start_penaltey
                else: 
                    D[i][j] = M[i-1][j]+extention_penalty
            #fill out the rest of the matrix
            else:
                D[i,j]=max(D[i,j-1]+extention_penalty,M[i,j-1]+start_penaltey)
    # #start the gap in seq2 matrix
    I = np.zeros((len(seq1)+1,len(seq2)+1))
    for i in range(len(seq1)+1):
        for j in range(len(seq2)+1):
            if i==0 and j==0:
                    I[i,j] = 0
            elif i==0:
                if j == 1: 
                    I[i][j] = start_penaltey
                else: 
                    I[i][j] = I[i][j-1]+extention_penalty
            elif j==0:
                if i==1: 
                    I[i][j] = start_penaltey
                else: 
                    I[i][j] = I[i-1][j]+extention_penalty
            #fill out the rest of the matrix
            else:
                I[i,j]=max(I[i-1,j]+extention_penalty,M[i-1,j]+start_penaltey)
    #return the cost matrix
    S = np.zeros((len(seq1)+1,len(seq2)+1))
    for i in range(len(seq1)+1):
        for j in range(len(seq2)+1):
            S[i,j]=max(M[i,j],D[i,j],I[i,j])
    return S


def backtracking(seq1:str, seq2:str, match_cost:int, mismatch_cost:int, start_penaltey:int, extention_penalty:int, matrix:list):
    """ Backtracking algorithm for affine gap alignment. Using the cost matrix and the sequences, it returns the optimal alignment."""
    i, j = len(seq1), len(seq2)
    align1, align2 = '', ''
    while i>0 and j>0:
        score_current=matrix[i][j]
        score_diagonal=matrix[i-1][j-1]
        score_left=matrix[i][j-1]
        score_above=matrix[i-1][j]
        if score_current==score_diagonal+match_cost and seq1[i-1]==seq2[j-1]:
            align1+=seq1[i-1]
            align2+=seq2[j-1]
            i-=1
            j-=1
        elif score_current==score_diagonal+mismatch_cost and seq1[i-1]!=seq2[j-1]:
            align1+=seq1[i-1]
            align2+=seq2[j-1]
            i-=1
            j-=1
        elif score_current==score_left+start_penaltey:
            align1+='-'
            align2+=seq2[j-1]
            j-=1
        elif score_current==score_above+start_penaltey:
            align1+=seq1[i-1]
            align2+='-'
            i-=1
        elif score_current==score_left+extention_penalty:
            align1+='-'
            align2+=seq2[j-1]
            j-=1
        elif score_current==score_above+extention_penalty:
            align1+=seq1[i-1]
            align2+='-'
            i-=1
    while i>0:
        align1+=seq1[i-1]
        align2+='-'
        i-=1
    while j>0:
        align1+='-'
        align2+=seq2[j-1]
        j-=1
    align1=align1[::-1] #reverse
    align2=align2[::-1] #reverse    
    return align1, align2



[[ 0. -3. -5. -7. -9.]
 [-3.  2. -1. -3. -5.]
 [-5. -1. -1. -4. -6.]
 [-7. -3. -4.  1. -2.]
 [-9. -5. -6. -2. -2.]]
('ATCG', 'ACCC')


In [ ]:
def affine_gap_cost_matrix(seq1, seq2, substitution_matrix, start_penalty, extention_penalty):
    n = len(seq1) + 1
    m = len(seq2) + 1

    # Initialize the matrices
    M = np.zeros((n, m))
    D = np.zeros((n, m))
    I = np.zeros((n, m))

    # Fill in the first row and column
    for i in range(1, n):
        M[i][0] = start_penalty + (i - 1) * extention_penalty
        I[i][0] = start_penalty + (i - 1) * extention_penalty
    for j in range(1, m):
        M[0][j] = start_penalty + (j - 1) * extention_penalty
        D[0][j] = start_penalty + (j - 1) * extention_penalty

    # Fill in the rest of the matrices
    for i in range(1, n):
        for j in range(1, m):
            substitution_cost = substitution_matrix[seq1[i-1], seq2[j-1]]
            M[i][j] = max(M[i-1][j-1] + substitution_cost, D[i-1][j-1] + substitution_cost, I[i-1][j-1] + substitution_cost)
            D[i][j] = max(M[i][j-1] + start_penalty, D[i][j-1] + extention_penalty)
            I[i][j] = max(M[i-1][j] + start_penalty, I[i-1][j] + extention_penalty)

    return M



import numpy as np

def backtrack(v, w, substitution_matrix, alignment_matrix):
    n, m = len(v), len(w)
    v = '-' + v
    w = '-' + w
    backtrack_matrix = np.zeros((n+1, m+1), dtype=int)

    for i in range(1, n+1):
        for j in range(1, m+1):
            match = alignment_matrix[i-1, j-1] + substitution_matrix[v[i]][w[j]]
            delete = alignment_matrix[i-1, j] + substitution_matrix[v[i]]['-']
            insert = alignment_matrix[i, j-1] + substitution_matrix['-'][w[j]]
            alignment_matrix[i, j] = max(match, delete, insert)
            if alignment_matrix[i, j] == match:
                backtrack_matrix[i, j] = 0
            elif alignment_matrix[i, j] == delete:
                backtrack_matrix[i, j] = 1
            else:
                backtrack_matrix[i, j] = 2

    i, j = n, m
    v_aligned, w_aligned = '', ''
    while i > 0 and j > 0:
        if backtrack_matrix[i, j] == 0:
            v_aligned = v[i] + v_aligned
            w_aligned = w[j] + w_aligned
            i -= 1
            j -= 1
        elif backtrack_matrix[i, j] == 1:
            v_aligned = v[i] + v_aligned
            w_aligned = '-' + w_aligned
            i -= 1
        else:
            v_aligned = '-' + v_aligned
            w_aligned = w[j] + w_aligned
            j -= 1

    while i > 0:
        v_aligned = v[i] + v_aligned
        w_aligned = '-' + w_aligned
        i -= 1
    while j > 0:
        v_aligned = '-' + v_aligned
        w_aligned = w[j] + w_aligned
        j -= 1

    return alignment_matrix[n, m], v_aligned, w_aligned



STORMS ANSWER:
Let S[i,j], D[i,j], and I[i,j] be as defined on slide 11 in the slides AffineGapcost.pdf. Here is pseudocode for filling them in a non-recursive manner cf. the recursions. We assume that 'max' is defined such that max(someting, undef) = something.

S[0..n, 0..m] = undef
D[0..n, 0..m] = undef
I[0..n, 0..m] = undef

for i = 0 to n:
    for j = 0 to n:
        // Compute D[i,j]
        v1 = v2 = undef
        if i>0 and j>=0 then v1 = S[i-1,j]-(a+b)
        if i>1 and j>=0 then v2 = D[i-1,j]-a
        D[i,j] = max(v1,v2)
        
        // Compute I[i,j]
        v1 = v2 = undef
        if i>=0 and j>0 then v1 = S[i,j-1]-(a+b)
        if i>=0 and j>1 then v2 = I[i,j-1]-a
        I[i,j] = max(v1,v2)
	
        // Compute S[i,j]
        v1 = v2 = v3 = v4 = undef
        if i=0 and j=0 then v1 = 0
        if i>0 and j>0 then v2 = S[i-1,j-1] + s(A[i],B[j])
        if i>0 and j>=0 then v3 = D[i,j]
        if i>=0 and j>0 then v4 = I[i,j]
        S[i,j] = max(v1,v2,v3,v3)
    endfor
endfor

Exercise 2.4
Backtracking when using linear gapcost: The slides from the lecture (about global pairwise alignment with linear gapcost) shows a recursive algorithm for backtracking an optimal alignment (when the dynamic programming table has been computed). Describe a non-recursive algorithm for backtracking an optimal alignment. (You can also consider how to do backtracking, if you are allowed to store additional information in the dynamic programming table while computing it.)

LOOK ABOVE


STORMS ANSWER
Consider the pseudo code for RecurBackTrack(i,j) on slide 83 in the slides about global alignment. Here is one possible rewrite to a non-recursive function.

func IterativeBackTrack(A[1..n], B[1..m], T[0..n][0..m])
    i = n, j = m
    while ( i >= 0 or j >= 0 ):
        if (i > 0) and (j > 0) and T[i,j] == T[i-1,j-1] + cost(A[i], B[j]) then
            “output column (A[i], B[j])”
            i=i–1
            j=j-1
        else if (i > 0) and (j >= 0) and T[i,j] == T[i-1,j] + g then
            “output column(A[i], -)”
            i=i-1
        else if (i >= 0) and (j > 0) and T[i,j] == T[i,j-1] + g then
            “output column(-, B[j])”
            j=j-1
        endif
    endwhile
end

The table T is the dynamic programming table, where T[i,j] stores the optimal cost of an alignment A[1..i] and B[1..j]. If you in T[i,j] also stores whether the optimal alignment of A[1..i] and B[1..j] ends in a (1) substitution, (2) insertion, or (3) deletion column, e.g. as T[i,j].lastcolumn, then the above can be simplefied to:

func IterativeBackTrack(A[1..n], B[1..m], T[0..n][0..m])
    i = n, j = m
    while ( i >= 0 or j >= 0 ):
        if (T[i,j].lastcolumn == substitution then
            “output column (A[i], B[j])”
            i=i–1
            j=j-1
        else if (T[i,j].lastcolumn == insertion then
            “output column(A[i], -)”
            i=i-1
        else if (T[i,j].lastcolumn == deletion then
            “output column(-, B[j])”
            j=j-1
        endif
    endwhile
end

When you compute T[i,j], you of course also know the value of T[i,j].lastcolumn. Hence, computing and storing this additional information does not add to the asymptotical running time, but it of course adds a little bit how much space is needed for each cell in the dynamic programming table.


Exercise 2.5
Backtracking when using affine gapcost: Explain how to perform backtracking when using affine gapcost as explained in the lecture/book, i.e. when the three dynamic programming tables have been computed and you know the cost of an optimal alignment. How long does it take (in terms of n and m)? Can you do it using only the S-table (c.f. the slides)?

STORMS ANSWER:

See attached slides 15-17 in AffineGapcost.pdf (from the lecture), which shows a way to perform backtracking in time O(n+m) for general gapcost (and therefore also affine gapcost).


Exercise 2.6
Backtracking when using general gapcost: Explain how to perform backtracking when using general gapcost as explained in the lecture/book. How long does it take? Can you do it in time O(n+m) as is the case when using linear gapcost?

STORMS ANSWER:

See attached slides 15-17 in AffineGapcost.pdf (from the lecture), which shows a way to perform backtracking in time O(n+m) for general gapcost (and therefore also affine gapcost).


Exercise 2.7
An extension of the basic algorithm for global pairwise alignment: Sometimes one might be interested in alignments where only gaps of certain lengths are allowed. Design an algorithm for finding an optimal global alignment of A[1..n] and B[1..m] with linear gapcost, but where only gaps of length 3, 6, 9, ... (i.e. gaps with length a multiple of 3) are allowed. What is the running time and space consumption of your algorithm?

In [4]:
def pairwise_3x(seq1,seq2,score,gap): #only for len(seq1) % 3 == 0 and len(seq2) % 3 == 0
    c = np.full(len(seq1)+1,len(seq2)+1,float('-inf'))
    c[0,0] = 0
    for i in range(3,len(seq1)+1,3):
        c[i,0] = c[i-3,0] + gap*3
    for j in range(3,len(seq2)+1,3):
        c[0,j] = c[0,j-3] + gap*3

    for i in range(1,len(seq1)+1):
        for j in range(1,len(seq2)+1):
            c[i,j] = max(c[i-1,j-1] + score(seq1[i-1],seq2[j-1]),
                        c[i-3,j] + gap*3,
                        c[i,j-3] + gap*3)
    return c

STORMS ANSWER 


Let C(i,j) be the cost of an optimal alignment of A[1..i] and A[1..j] with linear gapcost g, where only gaps of length a multiplum of 3 are allowed. We can compute C(i,j) as:

C(0,0) = 0
C(i,j) = min {C(i-1,j-1)+cost(A[i],B[j] if i>0 and j>0,
              C(i-3,j) + 3*g if i>2 and j>=0,
              C(i,j-3) + 3*g if i>=0 and j>2}

If we implement using dynamic programming, then we can compute C(n,m) in time and space O(nm).



Exercise 2.8
Free end gaps: How fast can you compute an optimal alignment when gaps columns at the ends of an alignment are free, i.e. when the two leftmost gap columns gaps and the three right most gap columns in the below alignment are free. (In this setting, the optimal alignment should have maximum cost, why?)

- - A C A - - - C G C A
A T T - T C C A C - - -

STORMS ANSWER:
Let C(i,j) be the cost of an optimal alignment of A[1..i] and A[1..j] with linear gapcost g, where endgaps as defined are free. We can compute C(i,j) as:

C(0,0) = 0
C(i,j) = max {C(i-1,j-1)+cost(A[i],B[j] if i>0 and j>0,
              C(i-1,j) + g if i>0 and j>0,
              C(i-1,j)     if i>0 and (j=0 or j=m),
              C(i,j-1) + g if i>0 and j>0,
              C(i,j-1)     if (i=0 or i=n) and j>0}

If we implement using dynamic programming, then we can compute C(n,m) in time and space O(nm).
